# Setup

In [ ]:
%%capture
!pip install -qU langchain
!pip install -qU langchain-google-genai
!pip install -qU langchain-huggingface
!pip install -qU langchain-qdrant
!pip install -qU langchain-community
!pip install -qU langgraph
!pip install fastembed
!pip install datasets
!pip install -U "fsspec[http]==2024.10.0"

In [ ]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import torch
from tqdm import tqdm

from IPython.display import display

from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Components

In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_api_key')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [ ]:
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = userdata.get('langsmith_api_key')
os.environ["LANGSMITH_PROJECT"] = "DSDE-Project"

In [ ]:
qdrant_api_key = userdata.get('qdrant_api_key')

# Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! ls 'drive/MyDrive/DSDE-Project-2025/Final Project'

bangkok_traffy.csv  df_filtered.csv  full_col.csv	   minidf_notype.csv
DataEng.ipynb	    df_for_ds.csv    handred_thousand.csv  Spark.ipynb


In [ ]:
dataset_path = 'drive/MyDrive/DSDE-Project-2025/Final Project/'
ds = pd.read_csv(dataset_path + "handred_thousand.csv")

# Embedding Model

In [ ]:
"""
"KanisornPutta/CeltaVigoBert"
"clicknext/phayathaibert"
"KanisornPutta/TrentIsNotFuckingLeavingBERT"
"""

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'trust_remote_code': True}
embeddings = HuggingFaceEmbeddings(model_name="KanisornPutta/TrentIsNotFuckingLeavingBERT",model_kwargs=model_kwargs)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/145k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="Pongsasit/mod-th-cross-encoder-minilm")

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

## Data Preparation

In [ ]:
ds.shape

(100000, 18)

In [ ]:
ds

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,type_str,type_set
40000,2023-GTFKRU,{ถนน},"เขตพระโขนง,สำนักการโยธา กทม.,ศูนย์ก่อสร้างและบ...",ถนนชำรุด,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59116,13.68661",18c ถ. ริมทางรถไฟปากน้ำ แขวงบางจาก เขตพระโขนง ...,บางจาก,พระโขนง,กรุงเทพมหานคร,2023-02-27 09:29:13.330312+00,เสร็จสิ้น,5.0,0,2024-04-25 04:04:36.074288+00,ถนน,{'ถนน'}
40001,2022-K7KNLB,{กีดขวาง},"เขตหลักสี่,ผอ.เขตหลักสี่ (นางสมฤดี),กลุ่มกรุงเ...",รถขายของ กีดขวางทางเท้า,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.56129,13.89496",โรงงาน 2/185 ซอยเมืองทองแขวงทุ่งสองห้อง เขตหลั...,ทุ่งสองห้อง,หลักสี่,กรุงเทพมหานคร,2022-06-26 11:42:27.177277+00,เสร็จสิ้น,5.0,0,2022-07-03 07:28:10.674178+00,กีดขวาง,{'กีดขวาง'}
40002,AHDPDG,"{ทางเท้า,ร้องเรียน,ป้าย,ความปลอดภัย,ถนน,แสงสว่าง}","เขตดินแดง,สำนักการโยธา กทม.,ศูนย์เครื่องมือกล ...",ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.55596,13.77114",สำนักการโยธา ถนน สุขเกษม แขวงดินแดง ดินแดง กรุ...,ดินแดง,ดินแดง,จังหวัดกรุงเทพมหานคร,2024-07-19 02:56:58.815335+00,กำลังดำเนินการ,NaN,0,2024-08-27 03:09:03.05679+00,"ทางเท้า,ร้องเรียน,ป้าย,ความปลอดภัย,ถนน,แสงสว่าง","{'ถนน', 'แสงสว่าง', 'ร้องเรียน', 'ความปลอดภัย'..."
40003,LHWRNH,"{ถนน,คลอง}","เขตหนองแขม,ฝ่ายเทศกิจ เขตหนองแขม",‘กรุณาอ่านข้อมูลบรรยายประกอบการปักหมุด’\n* ปัญ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.34071,13.68991",เพชรเกษม 81 มาเจริญ หนองแขม หนองแขม กรุงเทพมหา...,หนองแขม,หนองแขม,จังหวัดกรุงเทพมหานคร,2023-01-27 13:36:40.307272+00,เสร็จสิ้น,NaN,0,2023-01-28 02:58:29.080071+00,"ถนน,คลอง","{'คลอง', 'ถนน'}"
40004,2023-EMQ62U,"{ถนน,ต้นไม้}","เขตสาทร,ฝ่ายเทศกิจ เขตสาทร",ชาวบ้านวางต้นไม้หน้าบ้านทับบนรางน้ำ ถนนก็เล็กอ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.52163,13.71065",141 ซอย วัดปรก 2 แขวง ทุ่งวัดดอน เขต สาทร กรุง...,ทุ่งวัดดอน,สาทร,กรุงเทพมหานคร,2023-08-26 07:27:36.559217+00,เสร็จสิ้น,5.0,1,2023-09-15 03:44:58.325514+00,"ถนน,ต้นไม้","{'ถนน', 'ต้นไม้'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2024-MR4W3C,{ทางเท้า},"เขตบางบอน,ฝ่ายเทศกิจ เขตบางบอน",ได้มีการขายของบนทางเท้าในพื้นที่ห้ามขายและสร้า...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.40473,13.66185",51 ถ. เอกชัย บางบอนใต้ เขตบางบอน กรุงเทพมหานคร...,บางบอนใต้,บางบอน,กรุงเทพมหานคร,2024-03-27 04:41:05.085883+00,เสร็จสิ้น,NaN,0,2024-03-28 05:19:28.48937+00,ทางเท้า,{'ทางเท้า'}
49996,2024-686U3W,{ท่อระบายน้ำ},"เขตจอมทอง,ฝ่ายโยธา เขตจอมทอง",ท่ออุดตันมีน้ำขัง อาจเป็นแหล่งบ่อเกิดยุงลาย,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.47535,13.70117",32/63 Chumchon Sinlapa Det แขวง บางค้อ เขตจอมท...,บางค้อ,จอมทอง,กรุงเทพมหานคร,2024-02-09 18:29:25.367102+00,เสร็จสิ้น,NaN,0,2024-02-10 07:27:34.579807+00,ท่อระบายน้ำ,{'ท่อระบายน้ำ'}
49997,2022-KD6NQN,{สายไฟ},"เขตตลิ่งชัน,สำนักงาน กสทช. (ศูนย์รับแจ้งปัญหา ...",สายไฟเเละสายสื่อสารในบริเวณหน้าบ้านมีการร่วงหล...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.42084,13.78393",31/55 ถนน บรมราชชนนี แขวง ฉิมพลี เขตตลิ่งชัน ก...,ฉิมพลี,ตลิ่งชัน,กรุงเทพมหานคร,2022-07-02 10:00:35.580273+00,รอรับเรื่อง,NaN,0,2022-07-02 12:15:40.602967+00,สายไฟ,{'สายไฟ'}
49998,2024-KZABDG,"{ความปลอดภัย,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",แจ้งถนนเป็นหลุมขนาด 30 x 70 ซม. ลึกประมาณ 10 ซ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59224,13.80580",39 ซอย ลาดพร้าววังหิน 16 แขวงลาดพร้าว เขตลาดพร...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2024-10-03 01:17:53.368986+00,เสร็จสิ้น,5.0,0,2024-10-08 07:58:37.179802+00,"ความปลอดภัย,ถนน","{'ถนน', '

In [ ]:
ds['comment'][4]

KeyError: 4

In [ ]:
ds['address'][4]

In [ ]:
ds['photo'][0]

In [ ]:
import re

def clean_comment(comment):

    if type(comment) != str :
      return ""

    # comment = ''.join([' ' if c.isdigit() else c for c in comment])
    comment = ''.join([c for c in comment if c.isalpha() or c.isspace() or '\u0E00' <= c <= '\u0E7F'])
    comment = comment.replace('\n', ' ')
    comment = comment.replace('\r', ' ')
    comment = re.sub(' +', ' ', comment)  # replaces multiple spaces with one

    return comment

In [ ]:
def seperate_text(text):
    if type(text) != str :
      return ""
    return text.replace(",", ", ")

In [ ]:
ds['type_str'] = ds['type_str'].apply(seperate_text)

In [ ]:
ds['type_str'][0]

KeyError: 0

In [ ]:
ds.shape

(10000, 18)

In [ ]:
ds.dropna(subset=['comment'], inplace=True)

In [ ]:
ds.shape

(9909, 18)

In [ ]:
ds['context'] = ds.apply(
    lambda row: f'ชนิด: {seperate_text(row["type_str"])} : "{clean_comment(row["comment"])}" ที่อยู่ "{clean_comment(row["address"])}"',
    axis=1
)

In [ ]:
ds.loc[4]

In [ ]:
ds['context'][4]

In [ ]:
from langchain_core.documents import Document

In [ ]:
all_docs = []

for index, row in ds.iterrows(): # Use iterrows() for index-safe iteration
    context = row["context"]
    problem_type = row["type_str"]  # Get values directly from the row
    address = row["address"]
    ticket_id = row["ticket_id"]
    comment = row["comment"]

    if context is not None:
        doc = Document(
            page_content=str(context),
            metadata={
                "problem_type": problem_type,
                "address": address,
                "ticket_id": ticket_id
            }
        )
        all_docs.append(doc)

print(len(all_docs))

9909


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(all_docs)

print(f"Split wiki abstract into {len(all_splits)} sub-documents.")

Split wiki abstract into 9946 sub-documents.


In [ ]:
all_splits[0]

Document(metadata={'problem_type': 'ต้นไม้', 'address': '29/3 ถนน เพชรพระราม แขวง บางกะปิ เขตห้วยขวาง กรุงเทพมหานคร 10310 ประเทศไทย', 'ticket_id': '2024-HVXWHY', 'start_index': 0}, page_content='ชนิด: ต้นไม้ : "กิ่งไม้บังกล้องวงจรปิด" ที่อยู่ " ถนน เพชรพระราม แขวง บางกะปิ เขตห้วยขวาง กรุงเทพมหานคร ประเทศไทย"')

## Vector Database
Embed your documents in a vector database that supports hybrid search. Also set the retrieval mode to hybrid search.

We will use `QdrantVectorStore` [Learn more here](https://python.langchain.com/api_reference/qdrant/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore). (You can use any vector DB that can do hybrid search)

In [ ]:
from tqdm import tqdm

In [ ]:
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

# Create a Qdrant client for local storage
# client = QdrantClient(":memory:")

client = QdrantClient(
    url="https://e9cb8b6d-71bb-4578-a198-17d9e21a831f.eu-central-1-0.aws.cloud.qdrant.io",
    api_key=qdrant_api_key
)

collection_name = "test_dsde"

# Create a collection with both dense and sparse vectors
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config={"dense": VectorParams(size=768, distance=Distance.COSINE)},
#     sparse_vectors_config={
#         "sparse": SparseVectorParams(index=models.SparseIndexParams(on_disk=False))
#     },
# )

qdrant = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)

print("Adding documents to Qdrant...")
for doc in tqdm(all_splits, desc="Uploading documents"):
    qdrant.add_documents(documents=[doc])  # Process one document at a time

query = "น้ำท่วม"
found_docs = qdrant.similarity_search(query)
found_docs

Adding documents to Qdrant...


Uploading documents: 100%|██████████| 9946/9946 [28:10<00:00,  5.89it/s]


[Document(metadata={'problem_type': 'น้ำท่วม', 'address': '3 ซอย พหลโยธิน 67/1 แขวงอนุสาวรีย์ เขตบางเขน กรุงเทพมหานคร 10220 ประเทศไทย', 'ticket_id': '2023-AVPTLW', 'start_index': 0, '_id': '15b7922c-dab0-4bf5-8b1f-81bddca365fb', '_collection_name': 'test_dsde'}, page_content='ชนิด: น้ำท่วม : "น้ำท่วม ซอยพหลโยธิน ตรงเซเว่น ความสูงระดับข้อเท้า น้ำท่วมมาแล้วน้อยกว่า วัน" ที่อยู่ " ซอย พหลโยธิน แขวงอนุสาวรีย์ เขตบางเขน กรุงเทพมหานคร ประเทศไทย"'),
 Document(metadata={'problem_type': 'น้ำท่วม', 'address': '7/565 ซอย หมู่บ้านบัวขาว 41 มีนบุรี กรุงเทพมหานคร 10510 ประเทศไทย', 'ticket_id': '2022-PEPA9U', 'start_index': 0, '_id': '1f9cf97a-ce8e-4be0-ae19-8e42814fdfd4', '_collection_name': 'test_dsde'}, page_content='ชนิด: น้ำท่วม : "น้ำท่วมเข้าบ้าน" ที่อยู่ " ซอย หมู่บ้านบัวขาว มีนบุรี กรุงเทพมหานคร ประเทศไทย"'),
 Document(metadata={'problem_type': 'น้ำท่วม', 'address': '54/331 ซอย พัฒนาการ 65 แยก 1 แขวงประเวศ เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย', 'ticket_id': '2024-7AZ3MY', 'start_index': 0

# Retrievers


In [ ]:
retriever = qdrant.as_retriever(search_kwargs={"k": 10})

In [ ]:
reranker = CrossEncoderReranker(model=model, top_n=10)
reranked_retriever = ContextualCompressionRetriever(
    base_compressor=reranker , base_retriever=retriever
)

Take a subset of the dataset to evaluate the MRR of the retrievers.

In [ ]:
#sample = ds.take(1000)

In [ ]:
test_query = "เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?"

In [ ]:
test_docs = retriever.get_relevant_documents(test_query)
print(f'query : {test_query}')
print('-'*30)
for doc in test_docs[:10] :
  print(f'- {doc.page_content}')

In [ ]:
test_docs_reranked = reranked_retriever.get_relevant_documents(test_query)
print(f'query : {test_query}')
print('-'*30)
for doc in test_docs_reranked[:10] :
  print(f'- {doc.page_content}')

In [ ]:
for doc in test_docs_reranked[:10] :
  print(f'- {doc.metadata.get("ticket_id")}')

# Retrieval Evaluation
Coming Soom

In [ ]:
from tqdm import tqdm

In [ ]:
def compute_mrr(retriever, sample, top_k=3):
    mrr = 0
    failure = 0
    for s in tqdm(sample, desc="Processing queries"):
        query = s["question"]
        context = s["context"]
        docs = retriever.get_relevant_documents(query)
        found = False
        for i, doc in enumerate(docs[:top_k]):
            if doc.page_content.lower() in context.lower():
                mrr += 1 / (i + 1)
                found = True
                break  # Stop once we find the first relevant document
        if not found:
              failure += 1
    return mrr , failure

In [ ]:
# no_rerank, no_rerank_failures = compute_mrr(retriever, sample, 3)

In [ ]:
# print(f"MRR of the retriever without a reranker: {no_rerank / len(sample):.4f}")

# print(f"Percentage of queries with no relevant doc in top 3 (no rerank): {100 * no_rerank_failures / len(sample):.2f}%")

In [ ]:
# rerank, rerank_failures = compute_mrr(reranked_retriever, sample, 3)

In [ ]:
# print(f"MRR of the retriever with a reranker: {rerank / len(sample):.4f}")

# print(f"Percentage of queries with no relevant doc in top 3 (rerank): {100 * rerank_failures / len(sample):.2f}%")

# Agentic RAG

In [ ]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [ ]:
from langchain_core.tools import tool

# @tool(response_format="content_and_artifact")
# def retrieve(query: str):
#     """Retrieve information related to a query from a vector database of Traffy Fondue Dataset."""
#     # retrieved_docs = qdrant.similarity_search(query, k=5)
#     retrieved_docs = reranked_retriever.get_relevant_documents(query)
#     serialized = "\n\n".join(
#         (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
#         for doc in retrieved_docs
#     )
#     return serialized, retrieved_docs

In [ ]:
@tool(response_format="content")
def retrieve(query: str):
    """Retrieve information related to a query from a vector database of Traffy Fondue Dataset."""
    retrieved_docs = reranked_retriever.get_relevant_documents(query)
    serialized = "\n\n".join(
        (

            f"ticket_id: {doc.metadata.get('ticket_id')}\n"
            f"ประเภท: {doc.metadata.get('problem_type')}\n"
            f"สถานที่: {doc.metadata.get('address')}\n"
            f"รายละเอียด: {doc.page_content}"
        )
        for doc in retrieved_docs
    )
    return serialized


In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableConfig

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
# The ToolNode is roughly analogous to:

# tools_by_name = {tool.name: tool for tool in tools}
# def tool_node(state: dict):
#     result = []
#     for tool_call in state["messages"][-1].tool_calls:
#         tool = tools_by_name[tool_call["name"]]
#         observation = tool.invoke(tool_call["args"])
#         result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
#     return {"messages": result}

tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer based on retrieved problem reports."""
    # Get retrieved ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format tool messages for structured data
    structured_entries = []
    for tool_msg in tool_messages:
        content = tool_msg.content  # This is the string returned from the tool
        structured_entries.append(content)

    docs_content = "\n\n".join(structured_entries)

    # System message prompt
    system_message_content = (
        "คุณเป็นผู้ช่วยที่เชี่ยวชาญในการตอบคำถามจากข้อมูลปัญหาที่ถูกรายงานผ่านระบบแจ้งปัญหา Traffy Fondue "
        "ข้อมูลแต่ละรายการจะประกอบด้วยประเภทของปัญหา, สถานที่ และรายละเอียดของปัญหา "
        "กรุณาตอบคำถามจากข้อมูลด้านล่าง ถ้าคุณไม่พบคำตอบที่ตรง ให้ตอบว่า 'ไม่พบข้อมูลที่เกี่ยวข้อง' "
        "เลือกเพียงข้อมูลที่มีความเกี่ยวข้องกับ คำถาม และ ตอบ ticket_id ที่เกี่ยวของมา ใน format ticket_id : _id1, _id2, ..."
        "สรุปคำตอบออกมา และกล่าวถึงข้อมูลที่น่าสนใจ\n\n"
        f"{docs_content}"
    )

    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]

    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run model
    response = llm.invoke(prompt)
    return {"messages": [response]}


In [ ]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

# display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "What do u know"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "เขตจตุจักรมีปัญหาเกี่ยวกับอะไรมากที่สุด?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "ขยะแถวปทุมวัน"

for step in graph.stream(
    {"messages": [
        {
          "role": "user",
          "content": input_message,
        }
    ]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "ถ้าเกิดปัญหาทางเท้าเสียหายในเขตบางนา โมเดลจะสามารถหาข้อมูลที่เกี่ยวข้องได้หรือไม่?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()
